In [1]:
import pandas as pd
import numpy as np

In [2]:
hps = pd.read_hdf('/home/rob/Research/msm_sensitivity_analysis/data/msms/hpsample_long.h5')

In [3]:
hps['id'] = np.arange(hps.shape[0])

In [4]:
hps['rank'] = hps.groupby(['feature__value','distances__scheme', 'distances__transform'], dropna=False)['id'].rank()

In [5]:
hps

,cluster__max_iter,cluster__stride,tica__dim,tica__lag,tica__kinetic_map,tica__stride,cluster__k,feature__value,dihedrals__which,distances__scheme,distances__transform,distances__steepness,distances__centre,id,rank
hp_ix,,,,,,,,,,,,,,,
0,1000,10,19,36,True,1,306,dihedrals,all,NaN,NaN,0.000000,0.000000,0,1.0
1,1000,10,4,93,True,1,251,distances,NaN,closest-heavy,logistic,18.519573,0.249640,1,1.0
2,1000,10,18,68,True,1,98,distances,NaN,closest-heavy,logistic,38.533821,0.230190,2,2.0
3,1000,10,4,94,True,1,441,distances,NaN,ca,logistic,32.429605,0.553260,3,1.0
4,1000,10,15,87,True,1,104,dihedrals,all,NaN,NaN,0.000000,0.000000,4,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1000,10,18,39,True,1,148,distances,NaN,closest-heavy,linear,0.000000,0.000000,9995,1223.0
9996,1000,10,16,22,True,1,244,distances,NaN,closest-heavy,logistic,48.001321,1.434237,9996,1301.0
9997,1000,10,13,6,True,1,127,dihedrals,all,NaN,NaN,0.000000,0.000000,9997,4955.0


In [6]:
def keep(x, n): 
    if x['feature__value'] == 'dihedrals': 
        return x['rank'] <= n
    if x['feature__value'] == 'distances': 
        if x['distances__transform'] == 'linear': 
            return x['rank'] <= n
        else: 
            return x['rank'] <= 2*n
        
    
df = hps.loc[hps.apply(lambda x: keep(x, 20), axis=1), ].copy()
print(df.shape)
df.groupby(['feature__value','distances__scheme', 'distances__transform'], dropna=False).count()

(140, 15)


cluster__max_iter  \
feature__value distances__scheme distances__transform                      
dihedrals      NaN               NaN                                  20   
distances      ca                linear                               20   
                                 logistic                             40   
               closest-heavy     linear                               20   
                                 logistic                             40   

                                                       cluster__stride  \
feature__value distances__scheme distances__transform                    
dihedrals      NaN               NaN                                20   
distances      ca                linear                             20   
                                 logistic                           40   
               closest-heavy     linear                             20   
                                 logistic                           40   

                                                       tica__dim  tica__lag  \
feature__value distances__scheme distances__transform                         
dihedrals      NaN               NaN                          20         20   
distances      ca                linear                       20         20   
                                 logistic                     40         40   
               closest-heavy     linear                       20         20   
                                 logistic                     40         40   

                                                       tica__kinetic_map  \
feature__value distances__scheme distances__transform                      
dihedrals      NaN               NaN                                  20   
distances      ca                linear                               20   
                                 logistic                             40   
               closest-heavy     linear                               20   
                                 logistic                             40   

                                                       tica__stride  \
feature__value distances__scheme distances__transform                 
dihedrals      NaN               NaN                             20   
distances      ca                linear                          20   
                                 logistic                        40   
               closest-heavy     linear                          20   
                                 logistic                        40   

                                                       cluster__k  \
feature__value distances__scheme distances__transform               
dihedrals      NaN               NaN                           20   
distances      ca                linear                        20   
                                 logistic                      40   
               closest-heavy     linear                        20   
                                 logistic                      40   

                                                       dihedrals__which  \
feature__value distances__scheme distances__transform                     
dihedrals      NaN               NaN                                 20   
distances      ca                linear                               0   
                                 logistic                             0   
               closest-heavy     linear                               0   
                                 logistic                             0   

                                                       distances__steepness  \
feature__value distances__scheme distances__transform                         
dihedrals      NaN               NaN                                     20   
distances      ca                linear                                  20   
                                 logistic                                40   
             

In [7]:
for col in df.columns:
    if 'float' in str(df[col].dtype):
        if 'distance' in col:
            pass
        else:
            df[col] = df[col].astype(int)

In [8]:
df.drop(labels=['id', 'rank'], inplace=True, axis=1)

In [9]:
df.to_hdf('/home/rob/Research/msm_sensitivity_analysis/data/msms/hpsample.h5', key='hyperparameters')

/tmp/ipykernel_26596/1417713966.py:1: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['feature__value', 'dihedrals__which', 'distances__scheme',
       'distances__transform'],
      dtype='object')]

  df.to_hdf('/home/rob/Research/msm_sensitivity_analysis/data/msms/hpsample.h5', key='hyperparameters')


In [10]:
df.dtypes

cluster__max_iter         int64
cluster__stride           int64
tica__dim                 int64
tica__lag                 int64
tica__kinetic_map          bool
tica__stride              int64
cluster__k                int64
feature__value           object
dihedrals__which         object
distances__scheme        object
distances__transform     object
distances__steepness    float64
distances__centre       float64
dtype: object

In [11]:
df.head()

,cluster__max_iter,cluster__stride,tica__dim,tica__lag,tica__kinetic_map,tica__stride,cluster__k,feature__value,dihedrals__which,distances__scheme,distances__transform,distances__steepness,distances__centre
hp_ix,,,,,,,,,,,,,
0,1000,10,19,36,True,1,306,dihedrals,all,NaN,NaN,0.000000,0.00000
1,1000,10,4,93,True,1,251,distances,NaN,closest-heavy,logistic,18.519573,0.24964
2,1000,10,18,68,True,1,98,distances,NaN,closest-heavy,logistic,38.533821,0.23019
3,1000,10,4,94,True,1,441,distances,NaN,ca,logistic,32.429605,0.55326
4,1000,10,15,87,True,1,104,dihedrals,all,NaN,NaN,0.000000,0.00000
